In [17]:
import pandas as pd
import pyodbc
import csv

In [19]:
df = pd.read_csv('Walmart_Sales.csv')

In [5]:
# Validar datos nulos
if df.isnull().values.any():
    print("Hay valores nulos en el archivo CSV.")
else:
    print("No hay valores nulos en el archivo CSV.")

# Validar tipos de datos
try:
    df['Store'] = df['Store'].astype(int)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Weekly_Sales'] = df['Weekly_Sales'].astype(float)
    df['Holiday_Flag'] = df['Holiday_Flag'].astype(int)
    df['Temperature'] = df['Temperature'].astype(float)
    df['Fuel_Price'] = df['Fuel_Price'].astype(float)
    df['CPI'] = df['CPI'].astype(float)
    df['Unemployment'] = df['Unemployment'].astype(float)
    print("Los tipos de datos son correctos.")
except ValueError as e:
    print(f"Error en la conversión de tipos de datos: {e}")

# Validar rangos de valores
if (df['Holiday_Flag'].isin([0, 1]).all() and
    df['Weekly_Sales'].min() >= 0 and
    df['Fuel_Price'].min() >= 0 and
    df['CPI'].min() >= 0 and
    df['Unemployment'].min() >= 0):
    print("Los valores están dentro de los rangos esperados.")
else:
    print("Hay valores fuera de los rangos esperados.")



No hay valores nulos en el archivo CSV.
Error en la conversión de tipos de datos: time data "19-02-2010" doesn't match format "%m-%d-%Y", at position 2. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
Los valores están dentro de los rangos esperados.


In [7]:
# Convertir la columna Date a tipo de dato fecha especificando el formato
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Verificar la conversión
print(df.dtypes)


Store                    int64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object


In [14]:
# Conectar a la base de datos SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-Q0M1776;'
    'DATABASE=Automat_DB;'
    'UID=sa;'
    'PWD=gabdas'
)

# Crear un cursor
cursor = conn.cursor()

# Crear la tabla en SQL Server
create_table_query = '''
CREATE TABLE dbo.Walmart_Sales (
    Store INT,
    Date DATE,
    Weekly_Sales DECIMAL(10, 2),
    Holiday_Flag BIT,
    Temperature DECIMAL(5, 2),
    Fuel_Price DECIMAL(5, 2),
    CPI DECIMAL(5, 2),
    Unemployment DECIMAL(5, 2)
);
'''
cursor.execute(create_table_query)

# Leer el archivo CSV e insertar los datos en la tabla
with open(r'C:\Users\Usuario\Desktop\Auto.P\Walmart_Sales.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Saltar la cabecera
    for row in reader:
        cursor.execute(
            "INSERT INTO dbo.Walmart_Sales (Store, Date, Weekly_Sales, Holiday_Flag, Temperature, Fuel_Price, CPI, Unemployment) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]
        )

# Confirmar los cambios
conn.commit()

# Cerrar la conexión
cursor.close()
conn.close()
